# https://github.com/alphaj-jaeminyx/DeepAR-keras/blob/master/DeepAR.ipynb



In [3]:
import tensorflow as tf
import tensorflow_probability as tfp
from keras.layers import Input, Dense, LSTM, Dropout, TimeDistributed, Lambda

# LSTM To estimate the Gaussian distribution based on Simple DeepAR
# If you know the principle, you can make anything..

# challenge
# 1. convert to Seq2Seq 
# 2. LSTM introduction of a model other than (CNN-LSTM, GRU)
# 3. Try changing the distribution function => change the distribution estimation coefficient mu, sigma It is not and needs to be changed to a different value.
# 2주 정도 시간 들여서 해보기. (6주차에 솔루션 업로드)
# 4. 분포 함수 자체를 딥러닝으로 모델링 하기.  (5,6주차 때 다룰 예정, GAN/AutoEncoder)

class DeepAR(tf.keras.models.Model):
    def __init__(self, lstm_units, n_steps_in, n_steps_out, n_features):
        super().__init__()

        self.lstm = tf.keras.layers.LSTM(lstm_units, return_sequences=True, return_state=True, input_shape=(n_steps_in, n_features))
        self.dense_mu = tf.keras.layers.Dense(1) # 정규분포 (가우시안 분포)의 평균
        self.dense_sigma = tf.keras.layers.Dense(1, activation='softplus') # 정규분포 (가우시안 분포)의 시그마

    def call(self, inputs, initial_state=None):
        outputs, state_h, state_c = self.lstm(inputs, initial_state=initial_state)

        mu = self.dense_mu(outputs) # 정규분포의 평균을 추정하는 레이어
        sigma = self.dense_sigma(outputs) # 정규분포의 표준편차를 추정하는 레이어
        state = [state_h, state_c]

        return [mu, sigma, state]

# 모델의 로스를 계산하는 부분,
def log_gaussian_loss(mu, sigma, y_true):
    """
    Gaussian loss function
    """
                              # 모델링된 mu와 sigma를 가지고 분포 생성
                                                                    # y_true=실제값과 분포에서 샘플링된 값의 차이를 구함.
    return -tf.reduce_sum(tfp.distributions.Normal(loc=mu, scale=sigma).log_prob(y_true))


In [ ]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('/content/drive/MyDrive/시계열_교안/3주차/Gemini_ETHUSD_d.csv',  skiprows=1, parse_dates=True, index_col='Date')
df = df.sort_index().drop(['Symbol','Unix Timestamp'] , axis=1)

test_cutoff_date = df.index.max() - timedelta(days=90)

df_test = df[df.index > test_cutoff_date]['Close']
df_train = df[df.index <= test_cutoff_date]['Close']


############# 데이터를 정규화
training_mean = df_train.mean() # train의 평균
training_std = df_train.std() # train의 편차
df_training_value = (df_train - training_mean) / training_std # 데이터 = (원본 - 평균) / 편차로 정규화

test_mean = df_test.mean() # test의 평균
test_std = df_test.std() # test의 편차
df_test_value = (df_test - test_mean) / test_std # 데이터 = (원본 - 평균) / 편차로 정규화


##############

from numpy import array

def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		if out_end_ix > len(sequence):
			break
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

raw_seq = df_training_value
n_steps_in = 30
n_steps_out = 1

X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)

n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
y = y.reshape((y.shape[0], y.shape[1], n_features))

test_raw_seq = df_test_value

test_X, test_y = split_sequence(test_raw_seq, n_steps_in, n_steps_out)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], n_features))




In [ ]:
LSTM_UNITS = 128
EPOCHS = 100

model = DeepAR(LSTM_UNITS, n_steps_in, n_steps_out, n_features)

# optmizer
optimizer = tf.keras.optimizers.Adam()


# metric
rmse = tf.keras.metrics.RootMeanSquaredError()

def train_step(x, y):
    with tf.GradientTape() as tape:
        mu, sigma, _ = model(x)
        loss = log_gaussian_loss(mu, sigma, y) # Forward Learning
    # backword
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    rmse(y, mu)

for epoch in range(EPOCHS):
    train_step(X, y)
    print('Epoch %d, RMSE %.4f' % (epoch + 1, rmse.result().numpy()))
    rmse.reset_states()


In [ ]:


# 샘플링된 값들을 예측값으로 반환.
pred = model.predict(test_X)



In [ ]:
import numpy as np
lower_bound = []
upper_bound = []
median = []
for step_pred in pred[0]:
                           # 정규화된 값을 원래대로.
  lb = (np.quantile(step_pred, 0.05) + test_mean) * test_std
  ub = (np.quantile(step_pred, 0.95) + test_mean) * test_std
  med = (np.quantile(step_pred, 0.5) + test_mean) * test_std
  lower_bound.append(lb)
  upper_bound.append(ub)
  median.append(med)


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(111)


df['Close'].plot(ax=ax)
ax.vlines('2020-08-22', 0, 1000, linestyle='--', color='r', label='forecast boundary')
ax.fill_between(df_test[30:].index, lower_bound, upper_bound, color='b', alpha=0.1, label='95% Confidence Interval')
ax.plot(df_test[30:].index, median, label='Prediction')
ax.legend(loc='upper left')
# plt.suptitle(f"ARIMA {optimal[0][0]} Prediction Result (r2 score: {r2}")
# plt.show()